# Cap 12 Introdução às bibliotecas de modelagem em Python

## 12.1 Interface entre o pandas e o código dos modelos

In [67]:
import pandas as pd
import numpy as np

np.set_printoptions(precision=4)


In [68]:
data = pd.DataFrame(
    {
        "x0": [1, 2, 3, 4, 5],
        "x1": [0.01, -0.01, 0.25, -4.1, 0.0],
        "y": [-1.5, 0.0, 3.6, 1.3, -2.0],
    }
)


In [69]:
data


,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [70]:
data.columns


Index(['x0', 'x1', 'y'], dtype='object')

In [71]:
# Método to_numpy
data.to_numpy()


array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [72]:
# Para fazer a conversão de volta para um DataFrame
df2 = pd.DataFrame(data.to_numpy(), columns=["one", "two", "three"])


In [73]:
df2


,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [74]:
df3 = data.copy()


In [75]:
df3["strings"] = ["a", "b", "c", "d", "e"]


In [76]:
df3


,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [77]:
# dados heterogêneos, o resultado será um ndarray de objetos Python
df3.to_numpy()


array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [78]:
# loc com to_numpy
model_cols = ["x0", "x1"]


In [79]:
data.loc[:, model_cols].to_numpy()


array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [80]:
data["category"] = pd.Categorical(
    ["a", "b", "a", "a", "b"], categories=["a", "b"]
)


In [81]:
data


,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [82]:
# Criação de variáveis dummy a partir de dados categóricos
dummies = pd.get_dummies(data.category, prefix="category", dtype="Int64")


In [83]:
data_with_dummies = data.drop("category", axis=1).join(dummies)


In [84]:
data_with_dummies


,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## 12.2 Criação de descrições de modelos com o Patsy

In [85]:
data = pd.DataFrame(
    {
        "x0": [1, 2, 3, 4, 5],
        "x1": [0.01, -0.01, 0.25, -4.1, 0.0],
        "y": [-1.5, 0.0, 3.6, 1.3, -2.0],
    }
)


In [86]:
data


,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [87]:
import patsy


In [88]:
y, X = patsy.dmatrices("y ~ x0 + x1", data)


In [89]:
y


DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [90]:
X


DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [91]:
np.asarray(y)


array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [92]:
np.asarray(X)


array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [93]:
# Suprimindo o termo Intercept
patsy.dmatrices("y ~ x0 + x1 + 0", data)[1]


DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [94]:
coef, resid, _, _ = np.linalg.lstsq(X, y)


In [95]:
coef


array([[ 0.3129],
       [-0.0791],
       [-0.2655]])

In [96]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)


In [97]:
coef


Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### Transformações de dados em fórmulas do Patsy

In [98]:
y, X = patsy.dmatrices("y ~ x0 + np.log(np.abs(x1) + 1)", data)


In [99]:
X


DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [100]:
# Padronização e Centralização
y, X = patsy.dmatrices("y ~ standardize(x0) + center(x1)", data)


In [101]:
X


DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [102]:
# Função build_design_matrices
new_data = pd.DataFrame(
    {"x0": [6, 7, 8, 9], "x1": [3.1, -0.5, 0, 2.3], "y": [1, 2, 3, 4]}
)


In [103]:
new_X = patsy.build_design_matrices([X.design_info], new_data)


In [104]:
new_X


[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [105]:
# Somar colunas de um conjunto de dados pelo nome, deve-se encapsulá-las na
# função especial I
y, X = patsy.dmatrices("y ~ I(x0 + x1)", data)


In [106]:
X


DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

### Dados categóricos e o Patsy

In [107]:
data = pd.DataFrame(
    {
        "key1": ["a", "a", "b", "b", "a", "b", "a", "b"],
        "key2": [0, 1, 0, 1, 0, 1, 0, 0],
        "v1": [1, 2, 3, 4, 5, 6, 7, 8],
        "v2": [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7],
    }
)


In [108]:
y, X = patsy.dmatrices("v2 ~ key1", data)


In [109]:
X


DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [110]:
y, X = patsy.dmatrices("v2 ~ key1 + 0", data)


In [111]:
X


DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [112]:
# Função C para interpretar colunas numéricas como categóricas
y, X = patsy.dmatrices("v2 ~ C(key2)", data)


In [113]:
X


DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [114]:
data["key2"] = data["key2"].map({0: "zero", 1: "one"})


In [115]:
data


,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [116]:
y, X = patsy.dmatrices("v2 ~ key1 + key2", data)


In [117]:
X


DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [118]:
y, X = patsy.dmatrices("v2 ~ key1 + key2 + key1:key2", data)


In [119]:
X


DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

## 12.3 Introdução ao statsmodels

### Estimativa de modelos lineares

In [120]:
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [121]:
rng = np.random.default_rng(seed=12345)


def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = (size,)
    return mean + np.sqrt(variance) * rng.standard_normal(*size)


N = 100
X = np.c_[
    dnorm(0, 0.4, size=N),
    dnorm(0, 0.6, size=N),
    dnorm(0, 0.2, size=N),
]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps


In [122]:
X[:5]


array([[-0.9005, -0.1894, -1.0279],
       [ 0.7993, -1.546 , -0.3274],
       [-0.5507, -0.1203,  0.3294],
       [-0.1639,  0.824 ,  0.2083],
       [-0.0477, -0.2131, -0.0482]])

In [ ]:
y[:5]


array([-0.5995, -0.5885,  0.1856, -0.0075, -0.0154])

In [ ]:
# função sm.add_constant
X_model = sm.add_constant(X)


In [ ]:
X_model[:5]


array([[ 1.    , -0.9005, -0.1894, -1.0279],
       [ 1.    ,  0.7993, -1.546 , -0.3274],
       [ 1.    , -0.5507, -0.1203,  0.3294],
       [ 1.    , -0.1639,  0.824 ,  0.2083],
       [ 1.    , -0.0477, -0.2131, -0.0482]])

In [126]:
model = sm.OLS(y, X)


In [ ]:
results = model.fit()


In [ ]:
results.params


array([0.0668, 0.268 , 0.4505])

In [ ]:
print(results.summary())


                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.469
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                              28.51
Date:                qui, 09 out 2025   Prob (F-statistic):                    2.66e-13
Time:                        10:36:39   Log-Likelihood:                         -25.611
No. Observations:                 100   AIC:                                      57.22
Df Residuals:                      97   BIC:                                      65.04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [130]:
data = pd.DataFrame(X, columns=["col0", "col1", "col2"])


In [ ]:
data["y"] = y


In [ ]:
data[:5]


,col0,col1,col2,y
0,-0.900506,-0.189430,-1.027870,-0.599527
1,0.799252,-1.545984,-0.327397,-0.588454
2,-0.550655,-0.120254,0.329359,0.185634
3,-0.163916,0.824040,0.208275,-0.007477
4,-0.047651,-0.213147,-0.048244,-0.015374


In [133]:
results = smf.ols("y ~ col0 + col1 + col2", data=data).fit()


In [ ]:
results.params


Intercept   -0.020799
col0         0.065813
col1         0.268970
col2         0.449419
dtype: float64

In [ ]:
results.tvalues


Intercept   -0.652501
col0         1.219768
col1         6.312369
col2         6.567428
dtype: float64

In [ ]:
results.predict(data[:5])


0   -0.592959
1   -0.531160
2    0.058636
3    0.283658
4   -0.102947
dtype: float64

### Estimativas de processos de séries temporais

In [138]:
init_x = 4

values = [init_x, init_x]

N = 100

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)


In [ ]:
from statsmodels.tsa.ar_model import AutoReg


In [ ]:
MAXLAGS = 5


In [141]:
model = AutoReg(values, MAXLAGS)


In [ ]:
results = model.fit()


In [ ]:
results.params


array([ 0.0015,  0.6955, -0.3015, -0.143 ,  0.019 , -0.0588])

## 12.4 Introdução ao scikit-learn

In [ ]:
train = pd.read_csv("datasets/titanic/train.csv")


In [145]:
test = pd.read_csv("datasets/titanic/test.csv")


In [ ]:
train.head(4)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [ ]:
train.isna().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
test.isna().sum()


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [ ]:
impute_value = train["Age"].median()


In [150]:
train["Age"] = train["Age"].fillna(impute_value)


In [151]:
test["Age"] = test["Age"].fillna(impute_value)


In [152]:
train["IsFemale"] = (train["Sex"] == "female").astype(int)


In [153]:
test["IsFemale"] = (test["Sex"] == "female").astype(int)


In [154]:
predictors = ["Pclass", "IsFemale", "Age"]


In [ ]:
X_train = train[predictors].to_numpy()


In [ ]:
X_test = test[predictors].to_numpy()


In [ ]:
y_train = train["Survived"].to_numpy()


In [158]:
X_train[:5]


array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [ ]:
y_train[:5]


array([0, 1, 1, 1, 0])

In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
model = LogisticRegression()


In [162]:
model.fit(X_train, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [ ]:
y_predict = model.predict(X_test)


In [ ]:
y_predict[:10]


array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0])

In [ ]:
from sklearn.linear_model import LogisticRegressionCV


In [166]:
model_cv = LogisticRegressionCV(Cs=10)


In [167]:
model_cv.fit(X_train, y_train)


,Cs,10
,fit_intercept,True
,cv,None
,dual,False
,penalty,'l2'
,scoring,None
,solver,'lbfgs'
,tol,0.0001
,max_iter,100
,class_weight,None
,n_jobs,None


In [170]:
from sklearn.model_selection import cross_val_score


In [ ]:
model = LogisticRegression(C=10)


In [172]:
scores = cross_val_score(model, X_train, y_train, cv=4)


In [ ]:
scores


array([0.7758, 0.7982, 0.7758, 0.7883])